In [11]:
import re
import requests
from bs4 import BeautifulSoup
import re
import csv
import json
from pandas import DataFrame


# Wrangling the XML data

In [14]:
regex = re.compile(r'[\n\r\t]')
r = requests.get('http://www.dota2.com/hero/')
soup = BeautifulSoup(r.text, 'lxml')
links = soup.findAll('a', attrs={'href': re.compile("^http://www.dota2.com/hero/")})
#links = soup.findAll('a', attrs={'href': re.compile("^http://www.dota2.com/hero/earthshaker/")})
HeroList = []
for link in links:
    herolink = link.get('href')  
    r = requests.get(herolink)
    soup = BeautifulSoup(r.text, 'lxml')
    hero_name =soup.title.text.replace('Hero - ','')
    hero_img = soup.find('img', attrs={'src':re.compile("^https://cdn.cloudflare.steamstatic.com/apps/dota2/images/heroes/")})
    hero_bio = soup.find('div',attrs={'id':re.compile("^bioInner")})
    hero_roles = soup.find('p',attrs={'id':re.compile("^heroBioRoles")})
    hero_roles_list = hero_roles.text.split(" - ")
    hero_stats_intelligence = soup.find('div',attrs={'id':re.compile("^overview_IntVal")})
    hero_stats_agility = soup.find('div',attrs={'id':re.compile("^overview_AgiVal")})
    hero_stats_strength = soup.find('div',attrs={'id':re.compile("^overview_StrVal")})
    hero_stats_attack = soup.find('div',attrs={'id':re.compile("^overview_AttackVal")})
    hero_stats_speed = soup.find('div',attrs={'id':re.compile("^overview_SpeedVal")})
    hero_stats_defense = soup.find('div',attrs={'id':re.compile("^overview_DefenseVal")})
    hero_stats_level =[]
    hero_stats_hit_points =[]
    hero_stats_mana = []
    hero_stats_damage=[]
    hero_stats_armor=[]
    hero_stats_sight_range=''
    hero_stats_attack_range=''
    hero_stats_missile_speed=''
    stats_by_level_stat_row_colW = soup.findAll('div', attrs={'class':re.compile("^statRowCol")})
    for index, stats_by_level_stat_row_col in enumerate(stats_by_level_stat_row_colW):
        if (index<=2):
            hero_stats_level.append(stats_by_level_stat_row_col.text)
        elif(index>2 and index<=5):
            hero_stats_hit_points.append(stats_by_level_stat_row_col.text)
        elif(index>5 and index<=8):
            hero_stats_mana.append(stats_by_level_stat_row_col.text)
        elif(index>8 and index<=11):
            hero_stats_damage.append(stats_by_level_stat_row_col.text)
        elif(index>11 and index<=14):
            hero_stats_armor.append(stats_by_level_stat_row_col.text)
        elif(index==15):
            hero_stats_sight_range = stats_by_level_stat_row_col.text
        elif(index==16):
            hero_stats_attack_range=stats_by_level_stat_row_col.text
        elif(index==17):
            hero_stats_missile_speed=stats_by_level_stat_row_col.text
            
    Hero = {"hero_name":regex.sub("", hero_name), 
                 "image":hero_img.get("src"), "bio":regex.sub("", hero_bio.text),
                 "roles":hero_roles_list,
                  "stats":{
                    "intelligence": regex.sub("", hero_stats_intelligence.text),
                      "agility": regex.sub("", hero_stats_agility.text),
                      "strength": regex.sub("", hero_stats_strength.text),
                      "attack": regex.sub("", hero_stats_attack.text),
                      "speed": regex.sub("", hero_stats_speed.text),
                      "defense": regex.sub("", hero_stats_defense.text)},
                    "statsByLevel": {
                      "level": hero_stats_level,
                      "hit_points": hero_stats_hit_points,
                      "mana"  : hero_stats_mana,
                      "damage": hero_stats_damage,
                      "armor": hero_stats_armor,
                      "sight_range": hero_stats_sight_range,
                      "attack_range": hero_stats_attack_range,
                      "missile_speed": hero_stats_missile_speed},
                    "abilitis":[]
            }
    
    hero_abilities_names = soup.select('div.abilityHeaderRowDescription>h2')
    for hero_abilities_name in hero_abilities_names:
        Hero["abilitis"].append({"name":regex.sub("", hero_abilities_name.text)})  
    hero_abilities_descriptions = soup.select('div.abilityHeaderRowDescription>p')
    for index,hero_abilities_description in enumerate(hero_abilities_descriptions):
        Hero["abilitis"][index]["description"] = regex.sub("", hero_abilities_description.text)
    hero_abilities_images = soup.select('div.abilityIconHolder2>img')
    for index, hero_abilities_image in enumerate(hero_abilities_images):
        Hero["abilitis"][index]["image"] = hero_abilities_image.get("src")
    hero_abilities_videos = soup.select('div.abilityVideoContainer>iframe')
    for index, hero_abilities_video in enumerate(hero_abilities_videos):
        Hero["abilitis"][index]["video"] = hero_abilities_video.get("src")
    hero_mana_costs = soup.select('div.mana')
    for index, hero_mana_cost in enumerate(hero_mana_costs):
        Hero["abilitis"][index]["mana_cost"] = regex.sub("", hero_mana_cost.text)
    hero_cooldown = soup.select('div.cooldown')
    for index, hero_cool in enumerate(hero_cooldown):
        Hero["abilitis"][index]["cooldown"] = regex.sub("", hero_cool.text)
    hero_other_abilityFooterBox = soup.select('div.abilityFooterBox')
    for index, ability_footer in enumerate(hero_other_abilityFooterBox):
        other_ability_footer_dic={}
        ability_footer_span = ability_footer.findAll('span', attrs={'class':re.compile("^attribVal")})
        for last_a_ in ability_footer_span:
            key = str(last_a_.previous_sibling).replace(": ","").lower()
            key = key.replace(" ","_")
            other_ability_footer_dic[regex.sub("",key)] = regex.sub("",last_a_.text)
        Hero["abilitis"][index]["others"] = other_ability_footer_dic
    HeroList.append(Hero)
   
    with open('dota2_hero_data.json', 'w') as outfile:
        json.dump(HeroList, outfile)

In [16]:
HeroList[0]

{'hero_name': 'Earthshaker',
 'image': 'https://cdn.cloudflare.steamstatic.com/apps/dota2/images/heroes/earthshaker_full.png?v=6314492?v=6314492',
 'bio': 'Like a golem or gargoyle, Earthshaker was one with the earth but now walks freely upon it. Unlike those other entities, he created himself through an act of will, and serves no other master. In restless slumbers, encased in a deep seam of stone, he became aware of the life drifting freely above him. He grew curious. During a season of tremors, the peaks of Nishai shook themselves loose of avalanches, shifting the course of rivers and turning shallow valleys into bottomless chasms. When the land finally ceased quaking, Earthshaker stepped from the settling dust, tossing aside massive boulders as if throwing off a light blanket. He had shaped himself in the image of a mortal beast, and named himself Raigor Stonehoof. He bleeds now, and breathes, and therefore he can die. But his spirit is still that of the earth; he carries its power 

In [13]:
df_HeroList = DataFrame(HeroList)
df_HeroList

,hero_name,image,bio,roles,stats,statsByLevel,abilitis
0,Earthshaker,https://cdn.cloudflare.steamstatic.com/apps/do...,"Like a golem or gargoyle, Earthshaker was one ...","[Melee, Support, Initiator, Disabler, Nuker]","{'intelligence': '16 + 1.80', 'agility': '12 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Fissure', 'description': 'Slams the..."
1,Sven,https://cdn.cloudflare.steamstatic.com/apps/do...,"Sven is the bastard son of a Vigil Knight, bor...","[Melee, Carry, Disabler, Initiator, Durable, N...","{'intelligence': '16 + 1.30', 'agility': '21 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Storm Hammer', 'description': 'Sven..."
2,Tiny,https://cdn.cloudflare.steamstatic.com/apps/do...,"Coming to life as a chunk of stone, Tiny's ori...","[Melee, Carry, Nuker, Pusher, Initiator, Durab...","{'intelligence': '17 + 2.20', 'agility': '0 + ...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Avalanche', 'description': 'Bombard..."
3,Kunkka,https://cdn.cloudflare.steamstatic.com/apps/do...,"As The Admiral of the mighty Claddish Navy, Ku...","[Melee, Carry, Support, Disabler, Initiator, D...","{'intelligence': '18 + 1.80', 'agility': '14 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Torrent', 'description': 'Summons a..."
4,Beastmaster,https://cdn.cloudflare.steamstatic.com/apps/do...,Karroch was born a child of the stocks. His mo...,"[Melee, Initiator, Disabler, Durable, Nuker]","{'intelligence': '16 + 1.90', 'agility': '18 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Wild Axes', 'description': 'Beastma..."
...,...,...,...,...,...,...,...
115,Outworld Destroyer,https://cdn.cloudflare.steamstatic.com/apps/do...,"One of a lordly and magisterial race, Harbinge...","[Ranged, Carry, Nuker, Disabler]","{'intelligence': '30 + 4.20', 'agility': '22 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Arcane Orb', 'description': 'Adds e..."
116,Shadow Demon,https://cdn.cloudflare.steamstatic.com/apps/do...,Among the sovereign Demons with explicit acces...,"[Ranged, Support, Disabler, Initiator, Nuker]","{'intelligence': '21 + 3.30', 'agility': '18 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Disruption', 'description': 'Banish..."
117,Visage,https://cdn.cloudflare.steamstatic.com/apps/do...,Perched atop the entrance to the Narrow Maze s...,"[Ranged, Support, Nuker, Durable, Disabler, Pu...","{'intelligence': '22 + 2.90', 'agility': '11 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Grave Chill', 'description': 'Visag..."
118,Winter Wyvern,https://cdn.cloudflare.steamstatic.com/apps/do...,"Like many great poets, Auroth just wants time ...","[Ranged, Support, Disabler, Nuker]","{'intelligence': '26 + 3.60', 'agility': '16 +...","{'level': ['25', '15', '1'], 'hit_points': ['2...","[{'name': 'Arctic Burn', 'description': 'Winte..."
